In [3]:
# Python lib
import sys
sys.path.append('/home/daniel/gitrepos/vision/references/detection')
import torch
from engine import train_one_epoch, evaluate, _get_iou_types
import utils as engine_utils
import utils
import transforms as T
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch.utils.data
from PIL import Image, ImageFile
import pandas as pd
from tqdm import tqdm
import torch.nn as nn

import collections
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from torchvision import transforms
import train_utils
from glob import glob
from sklearn.model_selection import train_test_split

from hparams import create_hparams
import train
from coco_utils import get_coco_api_from_dataset
from coco_eval import CocoEvaluator

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [4]:
hparams = create_hparams()

tensorboard = train_utils.Tensorboard(hparams.checkpoint_path + "/logdir")
train_utils.prepare_checkpoint(hparams)
device = torch.device(f'cuda:{hparams.gpu_id}')

print("Loading data...")
train_loader, val_loader, label_encoder, num_classes = train.load_data(hparams)
# train_loader, label_encoder, num_classes = load_data(hparams)
print("Finished loading data")
print("Num classes: ", num_classes)
model_ft, optimizer, lr_scheduler = train.load_model(num_classes, device, hparams)

Loading data...
Batch size:  3
Finished encoding labels



  0%|          | 1014/1725611 [00:00<02:50, 10137.38it/s]

Train length: 1725611 Val length: 17431



  8%|▊         | 133055/1725611 [00:13<02:47, 9523.18it/s]


 15%|█▌        | 266204/1725611 [00:26<02:22, 10212.86it/s]


...

 83%|████████▎ | 1434118/1725611 [02:26<00:29, 10028.10it/s]


 91%|█████████ | 1565562/1725611 [02:39<00:15, 10336.66it/s]


 98%|█████████▊| 1697341/1725611 [02:52<00:02, 9938.27it/s]


100%|██████████| 17431/17431 [00:01<00:00, 9981.68it/s] 

Finished loading data
Num classes:  599


In [8]:
for i, (images, targets) in enumerate(train_loader):
    print(i)
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    print(train_loader.dataset[i])
    coco = get_coco_api_from_dataset(train_loader.dataset[i])
    break


  0%|          | 0/2 [00:00<?, ?it/s]

0
(tensor([[[0.0471, 0.0667, 0.1020,  ..., 0.5451, 0.3725, 0.2196],
         [0.0471, 0.0431, 0.0431,  ..., 0.5412, 0.3843, 0.3059],
         [0.0941, 0.0549, 0.0314,  ..., 0.5098, 0.3922, 0.3882],
         ...,
         [0.3255, 0.3373, 0.3490,  ..., 0.3373, 0.3529, 0.3843],
         [0.3255, 0.3529, 0.3686,  ..., 0.3412, 0.3451, 0.3765],
         [0.3490, 0.3647, 0.3882,  ..., 0.3216, 0.3451, 0.3882]],

        [[0.0588, 0.0824, 0.1255,  ..., 0.5529, 0.4039, 0.2824],
         [0.0784, 0.0588, 0.0627,  ..., 0.5804, 0.4353, 0.3451],
         [0.1333, 0.0745, 0.0510,  ..., 0.5804, 0.4510, 0.4275],
         ...,
         [0.2627, 0.2627, 0.2706,  ..., 0.1961, 0.2235, 0.2667],
         [0.2549, 0.2784, 0.2941,  ..., 0.1843, 0.2000, 0.2431],
         [0.2745, 0.2902, 0.3176,  ..., 0.1804, 0.1961, 0.2431]],

        [[0.0235, 0.0314, 0.0431,  ..., 0.4353, 0.3255, 0.2863],
         [0.0196, 0.0196, 0.0314,  ..., 0.3765, 0.3451, 0.2235],
         [0.0353, 0.0314, 0.0275,  ..., 0.4078, 0.3098,

ValueError: too many values to unpack (expected 2)

In [50]:
model_ft.train()
loss_dict = model_ft(images, targets)

In [ ]:
len(images)
images[0].shape

In [ ]:
print(len(targets))
targets[0]

In [ ]:
loss_dict

In [53]:
losses = sum(loss for loss in loss_dict.values())
loss_dict_reduced = engine_utils.reduce_dict(loss_dict)
losses_reduced = sum(loss for loss in loss_dict_reduced.values())
losses_reduced

tensor(9.5556, device='cuda:0', grad_fn=<AddBackward0>)

In [3]:
# to_delete = [model_ft, images, targets, loss_dict, losses]
# del to_delete

    
# torch.cuda.empty_cache()
# model_ft, optimizer, lr_scheduler = train.load_model(num_classes, device, hparams)
for images, targets in train_loader:
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    break
    
model_ft.eval()
outputs = model_ft(images)

In [4]:
iou_types = _get_iou_types(model_ft)

In [5]:
coco = get_coco_api_from_dataset(train_loader.dataset)

100%|██████████| 1783/1783 [00:22<00:00, 80.07it/s]

creating index...
index created!


In [17]:
coco_evaluator = CocoEvaluator(coco, iou_types)

In [18]:
print(coco_evaluator.coco_eval)

{'bbox': <pycocotools.cocoeval.COCOeval object at 0x7f7d836f9a90>}


In [19]:

res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
coco_evaluator.update(res)

In [20]:
# coco_evaluator.

In [21]:
coco_evaluator.synchronize_between_processes()

# accumulate predictions from all images
coco_evaluator.accumulate()
coco_evaluator.summarize()
# torch.set_num_threads(n_threads)

Accumulating evaluation results...
DONE (t=0.05s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=

In [23]:
coco_eval = coco_evaluator.coco_eval["bbox"]
params = coco_eval.params

In [24]:
coco_eval.stats[1]

0.0

In [48]:
def get_map( ap=1, iouThr=None, areaRng='all', maxDets=100 ):
    p = self.params
    iStr = ' {:<18} {} @[ IoU={:<9} | area={:>6s} | maxDets={:>3d} ] = {:0.3f}'
    titleStr = 'Average Precision' if ap == 1 else 'Average Recall'
    typeStr = '(AP)' if ap==1 else '(AR)'
    iouStr = '{:0.2f}:{:0.2f}'.format(p.iouThrs[0], p.iouThrs[-1]) \
        if iouThr is None else '{:0.2f}'.format(iouThr)

    aind = [i for i, aRng in enumerate(p.areaRngLbl) if aRng == areaRng]
    mind = [i for i, mDet in enumerate(p.maxDets) if mDet == maxDets]
    if ap == 1:
        # dimension of precision: [TxRxKxAxM]
        s = self.eval['precision']
        # IoU
        if iouThr is not None:
            t = np.where(iouThr == p.iouThrs)[0]
            s = s[t]
        s = s[:,:,:,aind,mind]
    else:
        # dimension of recall: [TxKxAxM]
        s = self.eval['recall']
        if iouThr is not None:
            t = np.where(iouThr == p.iouThrs)[0]
            s = s[t]
        s = s[:,:,aind,mind]
    if len(s[s>-1])==0:
        mean_s = -1
    else:
        mean_s = np.mean(s[s>-1])
    print(iStr.format(titleStr, typeStr, iouStr, areaRng, maxDets, mean_s))
    return mean_s

In [54]:
loss_dict

{'loss_box_reg': tensor(0.0070, device='cuda:0', grad_fn=<DivBackward0>),
 'loss_classifier': tensor(6.7118, device='cuda:0', grad_fn=<NllLossBackward>),
 'loss_objectness': tensor(1.7532, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 'loss_rpn_box_reg': tensor(1.0837, device='cuda:0', grad_fn=<DivBackward0>)}

In [56]:
train_loader.batch_size

3

In [59]:
results = model_ft(images, targets)

RuntimeError: CUDA out of memory. Tried to allocate 118.00 MiB (GPU 0; 10.91 GiB total capacity; 9.35 GiB already allocated; 29.62 MiB free; 152.30 MiB cached)

In [66]:
l = loss_dict['loss_classifier']

In [ ]:
l.